<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/247_PredRevenue_Gap_Orchestrator_Tier2_BizLogic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calcuate Recovery Probability

In [ ]:
def calculate_recovery_probability_score(
    gap: Dict[str, Any],
    revenue_baseline: Dict[str, Any]
) -> float:
    """
    Calculate recovery probability score (0-10 scale).

    Higher score = more likely to recover.

    Args:
        gap: Gap dictionary
        revenue_baseline: Revenue baseline data

    Returns:
        Score from 0-10
    """
    gap_type = gap.get("gap_type", "")
    weeks_at_risk = gap.get("weeks_at_risk", 0)
    revenue_trend = revenue_baseline.get("revenue_trend", "stable")

    # Base score on gap type and duration
    if gap_type == "zero_spend":
        # Zero spend is harder to recover from
        if weeks_at_risk >= 3:
            score = 2.0  # Very low recovery probability
        elif weeks_at_risk == 2:
            score = 4.0
        else:
            score = 6.0
    elif gap_type == "declining_revenue":
        # Declining revenue - check trend
        if revenue_trend == "declining":
            score = 5.0  # Moderate recovery probability
        else:
            score = 7.0
    elif gap_type == "below_baseline":
        # Below baseline - easier to recover
        score = 7.0
    else:
        score = 5.0  # Default

    # Adjust based on weeks at risk
    if weeks_at_risk > 4:
        score *= 0.7  # Reduce score for long-term gaps
    elif weeks_at_risk > 2:
        score *= 0.85

    return round(max(0.0, min(10.0, score)), 2)




# 🔍 **Function: `calculate_recovery_probability_score`**

**Purpose:**
Estimate **how likely a customer is to recover** from their gap *without heavy intervention*.

This is **not a prediction algorithm** — it’s a **decision heuristic** designed for orchestrators.

---

# 🧠 **What It’s Actually Doing (High-Level Insight)**

This function answers a crucial business question:

> “If a customer’s revenue dropped (or stopped), how likely are they to bounce back on their own?”

That’s *the* key variable needed for prioritizing action:

* **Low recovery probability → escalate immediately**
* **Medium recovery probability → targeted, moderate intervention**
* **High recovery probability → monitor or send light-touch nudge**

This turns raw data into **operational decision-making**.

---

# 🔬 **How the Logic Works (Simplified)**

### 1️⃣ First check *what type of gap it is*

Each gap type has different recovery patterns:

### **🟥 Zero spend gap → Very low recovery**

* Zero spend for 3+ weeks → almost lost
* 2 weeks → moderate risk
* 1 week → possible reactivation

This mirrors real retail/commerce behavior.

### **🟧 Declining revenue gap → Medium recovery**

* If the trend is still declining → lower recovery
* If spending stabilized → higher recovery

The function interprets “momentum.”

### **🟨 Below baseline gap → High recovery**

This is the classic “customer is a bit down, but still active.”
These are EASY to fix:

* Often due to temporary budget or preference shifts
* Small incentives usually restore spending

### **🟦 Unknown gaps → average probability**

Default = 5/10.

---

# 2️⃣ Then adjust probability based on **duration** (weeks at risk)

Longer problems are harder to fix.

* > 4 weeks → heavy penalty
* > 2 weeks → medium penalty

This mirrors customer lifecycle behavior:
**Recency predicts recovery.**

---

# 🧮 **What You Should Learn From This as a Data Scientist**

### ⭐ 1. This is *Decision Engineering*, not ML

The scoring is not about accuracy.
It’s about **ranking which customers need action first**.

### ⭐ 2. Humans naturally think in rules

Coaching sales reps or growth teams?
They already use heuristics like these — you’re turning intuition into code.

### ⭐ 3. Recovery probability is NOT a prediction

It’s a way to:

✨ Downweight lost causes
✨ Upweight recoverable customers
✨ Maximize ROI on interventions

### ⭐ 4. ML could replace this — but only if needed

This design allows you to:

* Drop in a classifier later
* Keep the decision system stable
* Run A/B tests comparing ML vs heuristic

---

# 🧩 **Why This Is Brilliant for Orchestrators**

Because it enables **multi-objective prioritization**:

* Gap severity
* Churn risk
* Revenue impact
* Customer value
* Recovery probability ← THIS FUNCTION

Together they answer:

> “Who should we act on first, and why?”

That’s the core of autonomous decision-making.

---

# 📌 **If I were training a Data Scientist on this…**

Here’s what I’d want you to observe:

### 💡 **1. Scoring systems reduce uncertainty**

Even if predictions were perfect (they’re not),
you STILL need a framework that decides:

* When to trigger an action
* How urgent it is
* Which action to take

### 💡 **2. Rules create trust**

Stakeholders understand:

* “Zero spend 3 weeks → low recovery”
* “10% below baseline → likely recoverable”

ML alone cannot replace this trust.

### 💡 **3. You can evolve this over time**

Want better accuracy?
Add:

* Logistic regression
* Gradient boosting
* Survival models
* LSTM or temporal transformers
* Bayesian updates

But keep the same **interface**.

Your orchestrator remains stable — your intelligence layer swaps in.



# Score Gap

In [ ]:
def score_gap(
    gap: Dict[str, Any],
    churn_risk_data: Optional[Dict[str, Any]],
    revenue_baseline: Dict[str, Any],
    all_customers_baselines: Dict[str, Dict[str, Any]],
    scoring_weights: Dict[str, float],
    max_gap_amount: float = 1000.0
) -> Dict[str, Any]:
    """
    Score a gap using all dimensions.

    Args:
        gap: Gap dictionary
        churn_risk_data: Churn risk data for customer
        revenue_baseline: Revenue baseline for customer
        all_customers_baselines: All customers' baselines
        scoring_weights: Weight configuration
        max_gap_amount: Maximum gap amount for normalization

    Returns:
        Gap dictionary with scores added
    """
    customer_id = gap.get("customer_id", "")

    # Calculate individual scores
    revenue_impact = calculate_revenue_impact_score(gap, max_gap_amount)
    churn_risk = calculate_churn_risk_score(gap, churn_risk_data)
    customer_value = calculate_customer_value_score(
        customer_id,
        revenue_baseline,
        all_customers_baselines
    )
    recovery_probability = calculate_recovery_probability_score(
        gap,
        revenue_baseline
    )

    # Calculate weighted final score
    final_score = (
        revenue_impact * scoring_weights.get("revenue_impact", 0.35) +
        churn_risk * scoring_weights.get("churn_risk", 0.30) +
        customer_value * scoring_weights.get("customer_value", 0.20) +
        recovery_probability * scoring_weights.get("recovery_probability", 0.15)
    )

    # Add scores to gap
    scored_gap = gap.copy()
    scored_gap.update({
        "revenue_impact_score": revenue_impact,
        "churn_risk_score": churn_risk,
        "customer_value_score": customer_value,
        "recovery_probability_score": recovery_probability,
        "final_score": round(final_score, 2)
    })

    return scored_gap




# ✅ **Understanding `score_gap()` — The Heart of Decision Engineering**

This is one of the most important functions in the entire orchestrator.

Why?

Because this is the moment when **raw analytical signals** → become **decision-ready intelligence**.

Let’s break it down.

---

# 🧠 **1. Inputs: What the function needs**

### **The function accepts multiple dimensions of intelligence:**

* The **gap object** (declining, below-baseline, zero spend)
* Optional **churn risk data** (if any)
* The customer’s **revenue baseline**
* **All customer baselines**, to compute customer value ranking
* **Scoring weights** (tunable)
* A normalization constant (`max_gap_amount`)

This design shows:

### → **Scoring is multi-dimensional, not single-metric.**

This is the foundation of modern decision engines.

---

# 🧩 **2. Step-by-step logic (what is computed)**

### **2.1 Revenue Impact**

```python
revenue_impact = calculate_revenue_impact_score(...)
```

How big is the revenue loss?

A \$400 gap on a \$50 customer is enormous.
A \$400 gap on a \$10,000 customer is trivial.

This captures the “size of the wound.”

---

### **2.2 Churn Risk**

```python
churn_risk = calculate_churn_risk_score(...)
```

How likely is the customer to churn?

A small gap with very high churn risk
is more important than a big gap with low churn risk.

---

### **2.3 Customer Value**

```python
customer_value = calculate_customer_value_score(...)
```

Ranks the customer against their peers.

This is essential because:

**Losing your top 10% customers is way worse
than losing your bottom 10%.**

---

### **2.4 Recovery Probability**

```python
recovery_probability = calculate_recovery_probability_score(...)
```

A clever addition.

A high-value customer
whose recovery potential is **high**
should be a **top priority**.

A customer with **low recovery potential**
moves lower in priority even if the gap is big.

---

# ⚖️ **3. Weighted Final Score (the decision engine)**

```python
final_score = (
    revenue_impact * w1 +
    churn_risk * w2 +
    customer_value * w3 +
    recovery_probability * w4
)
```

Default weights (can be tuned per business):

| Factor               | Weight | Why                                           |
| -------------------- | ------ | --------------------------------------------- |
| Revenue Impact       | 0.35   | Money matters most                            |
| Churn Risk           | 0.30   | Losing customers is worse than losing dollars |
| Customer Value       | 0.20   | VIPs get priority                             |
| Recovery Probability | 0.15   | Focus on winnable cases                       |

This is EXACTLY how real-world CRM prioritization works
in enterprise tools like Salesforce Einstein, Adobe Journey AI, etc.

Except—your orchestrator is transparent and modifiable.

---

# 🏁 **4. Output: Action-ready object**

The function returns a **fully enriched gap object**, including:

* Revenue impact score
* Churn risk score
* Customer value score
* Recovery probability score
* **final score (0–10)** → this is what the orchestrator uses to rank priorities

This transforms raw analytics → into **ranked business decisions.**

---

# 🎯 Why This Function Matters So Much

## ✔ It’s the “decision brain” of the orchestrator

Everything before this (baseline, trend, gap detection) is **signal generation**.
Everything after this (ranking, reporting, action) is **execution**.

This function is the **bridge**.

## ✔ It enables automated prioritization

Companies struggle to decide:

* Which customers do we reach out to first?
* What issues do we tackle today?
* Which accounts are bleeding money?

This function answers that automatically.

## ✔ It’s fully explainable

Every score is traceable and inspectable.
No AI hallucination.
No black box.

## ✔ It is 100% customizable

Businesses can tune:

* weights
* thresholds
* scoring formulas

And align it to real strategy.

---

# 🤖 What you should learn from this (as a future decision-engineer)

### **1. Multi-objective scoring**

Modern AI systems rarely optimize for **one** metric.
They balance:

* revenue
* risk
* customer value
* probability of success

This is the future of applied data science.

---

### **2. Decision frameworks > predictive models**

Predictive models are nice…
But prioritization engines **drive business outcomes**.

You now know how to build one.

---

### **3. Actionability as a design principle**

The goal is NOT to predict.

The goal is:

👉 **Detect → Score → Decide → Act**

This function is the embodiment of that philosophy.




# Score All Gaps

In [ ]:
def score_all_gaps(
    revenue_gaps: List[Dict[str, Any]],
    churn_risk_customers: List[Dict[str, Any]],
    customer_revenue_baseline: Dict[str, Dict[str, Any]],
    scoring_weights: Dict[str, float]
) -> List[Dict[str, Any]]:
    """
    Score all gaps.

    Args:
        revenue_gaps: List of all detected gaps
        churn_risk_customers: List of churn risk customers
        customer_revenue_baseline: Revenue baselines for all customers
        scoring_weights: Weight configuration

    Returns:
        List of scored gaps
    """
    # Build churn risk lookup
    churn_risk_lookup = {
        risk["customer_id"]: risk
        for risk in churn_risk_customers
    }

    # Find max gap amount for normalization
    if revenue_gaps:
        max_gap_amount = max(abs(gap.get("gap_amount", 0.0)) for gap in revenue_gaps)
        if max_gap_amount == 0:
            max_gap_amount = 1000.0  # Default
    else:
        max_gap_amount = 1000.0

    # Score each gap
    scored_gaps = []
    for gap in revenue_gaps:
        customer_id = gap.get("customer_id", "")
        revenue_baseline = customer_revenue_baseline.get(customer_id, {})
        churn_risk_data = churn_risk_lookup.get(customer_id)

        scored_gap = score_gap(
            gap,
            churn_risk_data,
            revenue_baseline,
            customer_revenue_baseline,
            scoring_weights,
            max_gap_amount
        )

        scored_gaps.append(scored_gap)

    return scored_gaps



# ⭐ Function: `score_all_gaps(...)` — What You Should Learn

This is the **master scoring loop** that takes every detected gap and enriches it with a full multi-dimensional score.

It embodies several core concepts of **decision engineering**, **orchestrator design**, and **applied analytics**.

Let’s break it down.

---

# 🧩 **1. It builds a lookup table for churn risk**

```python
churn_risk_lookup = {
    risk["customer_id"]: risk
    for risk in churn_risk_customers
}
```

### Purpose:

* Fast access: O(1) lookup to see if a given customer already has churn risk data.
* Compose signals: scoring combines *gap data* + *churn data*.
* Avoid re-computing churn multiple times.

### Concept to learn:

> **Orchestrators use lookups to combine multiple analytic streams into a single decision.**

---

# 📏 **2. It normalizes scores using the max gap amount**

```python
if revenue_gaps:
    max_gap_amount = max(abs(gap.get("gap_amount", 0.0)) for gap in revenue_gaps)
```

### Purpose:

* Revenue impact scoring needs a “ceiling”.
* This normalizes 0–10 impact scores across the entire dataset.

### Concept to learn:

> **Normalization is essential for multi-factor scoring systems.**
> Without normalization, one metric may dominate unfairly.

---

# 🎯 **3. It loops through every gap and computes full scores**

For each gap:

* revenue impact score
* churn risk score
* customer value score
* recovery probability score
* weighted final score

It calls:

```python
scored_gap = score_gap(...)
```

### Purpose:

To turn raw gap events into **prioritized, ranked, actionable insights**.

### Concept:

> **This is where raw analytics become decisions.**

This is not dashboards.
This is not predictions.
This is **decision automation**.

---

# 🧮 **4. It merges everything into a single scored list**

```python
scored_gaps.append(scored_gap)
```

### Purpose:

The output of this function becomes the input for **ranking → actions → report generation**.

### Concept:

> **The orchestrator builds up richer and richer intelligence at each stage.**

---

# 🚀 What You Should Learn From This Function

### ✔ 1. **Multi-dimensional scoring systems**

This function shows how to combine multiple types of signals into a unified score.

### ✔ 2. **Separation of concerns**

Scoring is separate from detection, analysis, and reporting.

Beautiful architecture discipline.

### ✔ 3. **Normalization strategies**

The system avoids arbitrary values by scaling relative to the dataset.

### ✔ 4. **Lookup-based enrichment**

Signals from different utilities (gap detection, churn detection, baseline analysis) get merged automatically.

### ✔ 5. **Agent design principle: scoring → ranking → decision**

This is the decision-engine pipeline:

1. Compute baseline metrics
2. Detect anomalies
3. Score gaps
4. Rank them
5. Generate reports / trigger actions

Everything is modular and swappable.

---

# 🧠 Data Scientist Takeaway

This function embodies the heart of **decision engineering**:

* turning data **→ signals**
* signals **→ scores**
* scores **→ priorities**
* priorities **→ automated decisions**

If you master this pattern, you can build **any business decision engine**:





# Rank Gaps

In [ ]:
def rank_gaps(
    scored_gaps: List[Dict[str, Any]],
    top_n: int = 20
) -> Dict[str, Any]:
    """
    Rank gaps by final score and select top N.

    Args:
        scored_gaps: List of scored gaps
        top_n: Number of top gaps to select

    Returns:
        Dictionary with ranked_gaps, top_priority_gaps, and summary
    """
    # Sort by final_score (descending)
    ranked = sorted(
        scored_gaps,
        key=lambda x: x.get("final_score", 0.0),
        reverse=True
    )

    # Select top N
    top_priority = ranked[:top_n]

    # Calculate summary metrics
    total_customers_analyzed = len(set(gap.get("customer_id") for gap in scored_gaps))
    customers_with_gaps = total_customers_analyzed
    total_revenue_gap = sum(abs(gap.get("gap_amount", 0.0)) for gap in scored_gaps)

    # Count by severity
    high_priority = sum(1 for gap in top_priority if gap.get("severity") == "high")
    medium_priority = sum(1 for gap in top_priority if gap.get("severity") == "medium")
    low_priority = sum(1 for gap in top_priority if gap.get("severity") == "low")

    # Estimate potential recovery revenue (sum of gaps for top N)
    potential_recovery_revenue = sum(abs(gap.get("gap_amount", 0.0)) for gap in top_priority)

    summary = {
        "total_customers_analyzed": total_customers_analyzed,
        "customers_with_gaps": customers_with_gaps,
        "total_revenue_gap": round(total_revenue_gap, 2),
        "high_priority_gaps": high_priority,
        "medium_priority_gaps": medium_priority,
        "low_priority_gaps": low_priority,
        "potential_recovery_revenue": round(potential_recovery_revenue, 2)
    }

    return {
        "ranked_gaps": ranked,
        "top_priority_gaps": top_priority,
        "gap_summary": summary
    }



# 🔥 **Function: `rank_gaps` — The “Decision Output Engine”**

This is where your orchestrator **finally decides what matters most**.

Everything before this was *signal creation*.
This function is *signal prioritization*.
This is the moment the agent says:

> “Out of everything I detected, here’s what actually deserves action.”

Let’s break down the logic:

---

# 🧠 1. **Sort gaps by their final score**

```python
ranked = sorted(
    scored_gaps,
    key=lambda x: x.get("final_score", 0.0),
    reverse=True
)
```

This takes every scored gap (0–10 scale) and orders them from highest → lowest.

### Why it matters:

This is how the orchestrator **decides priority**, replacing the human act of:

* Looking at dashboards
* Making subjective calls
* Ranking customers manually

The orchestrator now does that automatically.

---

# 🔍 2. **Select the top N most important issues**

```python
top_priority = ranked[:top_n]
```

If you set `top_n=20`, you get the **top 20 highest-value issues**.

### Why this is powerful:

You control the *focus window*. Instead of overwhelming teams:

> “Here are 3,000 anomalies…”

You get:

> “Here are the **20** most important ones — take action *here first*.”

---

# 📊 3. **Create summary metrics**

This block creates a business-friendly executive summary.

### Key metrics computed:

#### ✔ Unique customers with gaps

```python
total_customers_analyzed = len(set(gap.get("customer_id") for gap in scored_gaps))
```

#### ✔ Total revenue at risk

```python
total_revenue_gap = sum(abs(gap.get("gap_amount", 0.0)) for gap in scored_gaps)
```

#### ✔ Gap severity distribution

```python
high_priority = sum(1 for gap in top_priority if gap.get("severity") == "high")
medium_priority = ...
low_priority = ...
```

#### ✔ Potential revenue recovery

```python
potential_recovery_revenue = sum(abs(gap.get("gap_amount", 0.0)) for gap in top_priority)
```

### Why this is critical:

This is exactly what leaders want:

* “How bad is it?”
* “How urgent is it?”
* “What could we recover if we act now?”

This transforms raw data into **actionable business intelligence**.

---

# 🧾 4. **Return a structured result**

```python
return {
    "ranked_gaps": ranked,
    "top_priority_gaps": top_priority,
    "gap_summary": summary
}
```

This output becomes the **input to the report generator**, which turns it into:

* Executive summaries
* Top customer lists
* Action recommendations
* Financial impact estimates

The orchestrator is now able to communicate like an analyst.

---

# 🧠 Why This Function Is So Important

This is the **Decision Engineering core**:

It answers:

### ✔ What should we act on first?

### ✔ Which customers matter most?

### ✔ Where is the revenue risk?

### ✔ What will generate the highest return?

It turns all the upstream analytics into **a prioritized action list**, the heart of any intelligent business system.

---

# 🚀 What You Should Learn From This Function

### 1. **Prioritization is the core of every orchestrator**

All powerful AI systems must decide:

* What to ignore
* What to watch
* What to escalate
* What to act on

This function does exactly that.

---

### 2. **Weighted scoring → ranking → decisions**

This is the blueprint for decision automation:

1. Compute signals
2. Score signals
3. Rank signals
4. Focus attention
5. Trigger action

This will appear in EVERY high-value orchestrator you'll ever build.

---

### 3. **Business value is created here**

This step translates numbers into:

* \$ lost
* \$ at risk
* \$ recoverable
* Who to call
* What to fix

This function = direct business impact.

---

### 4. **You can easily plug in more dimensions**

The ranking logic doesn’t care what contributed to the score.

Add new detectors, new scores, new ML models →
As long as they output a score, this function will prioritize it.

That’s what makes your architecture so scalable.


